#### Side note for github readers: This notebook has been made to optimize the delivery of the project into a Graasp page. So please don't yell at me when you see wierd stuff e.g. multiple importation of the same library

# Project description
## Problem
Building a tool to find correlation between the new Covid cases and the Vaccination campaign in Switzerland, based on the official datasets provided by the Swiss Government.

## Datasets
- https://www.bag.admin.ch/bag/fr/home/krankheiten/ausbrueche-epidemien-pandemien/aktuelle-ausbrueche-epidemien/novel-cov/situation-schweiz-und-international.html
- https://www.bag.admin.ch/bag/fr/home/krankheiten/ausbrueche-epidemien-pandemien/aktuelle-ausbrueche-epidemien/novel-cov/impfen.html#-695394864
- https://github.com/openZH/covid_19
- https://github.com/rsalzer/COVID_19_VACC
- https://github.com/doerfli/foph-covid19-data
- https://github.com/daenuprobst/covid19-cases-switzerland
- https://github.com/covid-19-Re/dailyRe-Data

## Similar projects
The following project are all using data from the sars-cov 2 and display information but none is trying to correlate information together, and particularly in Switzerland.
- https://www.covid19.admin.ch/fr/overview
- https://www.covid19.admin.ch/fr/epidemiologic/vacc-doses
- https://ourworldindata.org/covid-vaccinations
- https://corona-data.ch/
- https://www.covid19-data.ch/vaccinations
- https://www.corona-immunitas.ch/fr/
- https://sciencetaskforce.ch/en/current-situation/

## Abstraction
As I am no expert in the medical field, I will try to gather as much information (that I think important initially) to build a dataset containing the following (preferably by day, by canton):

- number of tests done (useless value in the end)
- number of positive tests
- number of hospitalizations
- number of deaths
- number of vaccinations done
- number of vaccinations completed (seconde dose)
- percentage of success of the vaccin
- reproductive number Re
- percentage of vaccination

Also trying to define calculated data (could be completely wrong as I have no expertise)
- percentage of collective immunity, based on:
    - time estimation of the natural immunity post-sickness (Serological)
    - time estimation of vaccines
    - estimation of the percentage of success of the vaccines,
    - number of positive tests
    - number of deaths
    - reproductive number

Finally, I will be focusing on maintained (up to date) datasets:
- Will focus on datasets auto-maintained with scheduled scrapers
- Will not try to fill holes
- Will only display synchronized data until the latest dataset update

## Patterns
- use of a graph template to display data
- data loading process

## Decomposition
- use of pandas' dataframes for the data manipulations
- gather the official raw datasets from sources
    - download csv files ready to use (summaries)
    - make a scraper for csv files unready to use as it is
    - build a custom dataset for our project
    - standartize the timestamps, the cantons
    - add calculated values
- use of different charts and maps to highlight key information
    - plots
    - Switzerland map
- display key-values in tables


# Gather data
- It came to conclusion that it would not be necessary to build a scraper, as multiple public projects are already gathering and building datasets.
- After some research, we could find all the features required, but no dataset was aggregating all the required features at once.
- It was then undertaken to build a custom dataset by merging the minimum amount of datasets to cumulate our required features.
- 2 datasets were selected:
  - (Vaccins) foph-covid19-data: https://github.com/doerfli/foph-covid19-data/blob/main/vacc_data.csv

| Field Name            | Description                                                                              | Format     | Example    |
|-----------------------|------------------------------------------------------------------------------------------|------------|------------|
| date                  | date                                                                                     | YYYY-MM-DD | 2021-02-27 |
| canton                | two-letter name of canton or CH for whole switzerland or CHFL for whole CH+liechtenstein | Text       | NE         |
| deliveredTotal        | sum of vaccines delivered to the canton till the date                                    | Number     | 21000      |
| deliveredPer100       | sum of vaccines delivered to the canton till the date per 100 population                 | Number     | 11.9       |
| administeredTotal     | number of vaccines vaccinated in the canton till the date                                | Number     | 15955      |
| administeredPer100    | number of vaccines vaccinated in the canton till the date per 100 population             | Number     | 9.04       |
| fullyVaccinatedTotal  | number of people who are fully vaccinated (2 doses at the moment)                        | Number     | 2572       |
| fullyVaccinatedPer100 | number of people who are fully vaccinated (2 doses at the moment) per 100 population     | Number     | 1.46       |

  - (Positive Tests) covid19-cases-switzerland: https://github.com/daenuprobst/covid19-cases-switzerland/blob/master/covid19_cases_switzerland_openzh.csv

| Field Name                              | Description                                         | Format     | Example                |
|-----------------------------------------|-----------------------------------------------------|------------|------------------------|
| Date                                    | Date of notification                                | YYYY-MM-DD | 2020-02-26             |
| Cantons Abbreviations [AG to ZH and CH] | Total of Confirmed cases until this date                                    | Value      | 11.0                   |
| Canton_diff [AG to ZH and CH]           | Current date new cases                              | Value      | 5.0                    |
| Canton_pc [AG to ZH and CH]             | Percent cases of the canton population              | Value      | 1.2878435761777038e-06 |
| Canton_diff_pc [AG to ZH and CH]        | Current date percent cases of the canton population | Value      | 5.85383443717138e-07   |

  - (Deaths) covid19-cases-switzerland: https://github.com/daenuprobst/covid19-cases-switzerland/blob/master/covid19_fatalities_switzerland_openzh.csv

| Field Name                              | Description                                         | Format     | Example                |
|-----------------------------------------|-----------------------------------------------------|------------|------------------------|
| Date                                    | Date of notification                                | YYYY-MM-DD | 2020-02-26             |
| Cantons Abbreviations [AG to ZH and CH] | Total of Deaths until this date                                     | Value      | 0.0                   |
| Canton_diff [AG to ZH and CH]           | Current date new deaths                              | Value      | 0.0                    |
| Canton_pc [AG to ZH and CH]             | Percent deaths of the canton population              | Value      | 0.0 |
| Canton_diff_pc [AG to ZH and CH]        | Current date percent deaths of the canton population | Value      | 0.0   |
  
  - (Hospitalizations) covid19-cases-switzerland: https://github.com/daenuprobst/covid19-cases-switzerland/blob/master/covid19_hospitalized_total_switzerland_openzh.csv
 
| Field Name                              | Description                                         | Format     | Example                |
|-----------------------------------------|-----------------------------------------------------|------------|------------------------|
| Date                                    | Date of notification                                | YYYY-MM-DD | 2020-02-26             |
| region [AG to ZH and CH] | Total of Hospitalizations until this date                                     | Value      | 0.0                   |
| Canton_diff [AG to ZH and CH]           | Current date new Hospitalizations                              | Value      | 0.0                    |
| Canton_pc [AG to ZH and CH]             | Percent Hospitalization of the canton population              | Value      | 0.0 |
| Canton_diff_pc [AG to ZH and CH]        | Current date percent Hospitalization of the canton population | Value      | 0.0   |

  - (Re step-wise) dailyRe-Data: https://github.com/covid-19-Re/dailyRe-Data/blob/master/CHE-confCasesSWestimates.csv
  
| Field Name       | Description                            | Format     | Example    |
|------------------|----------------------------------------|------------|------------|
| region           | Canton abbreviation [AG to ZH and CHE] | string | CHE        |
| date             | Date of notification                   | YYYY-MM-DD | 2020-02-09 |
| median_R_mean    | Median R_e                             | Value      | 1.68       |
| median_R_highHPD | High Median R_e                        | Value      | 3.32       |
| median_R_lowHPD  | Low Median R_e                         | Value      | 0.59       |


In [2]:
import pandas as pd

df_vacc = pd.read_table("https://raw.githubusercontent.com/doerfli/foph-covid19-data/master/vacc_data.csv",sep=",")
df_cases = pd.read_table("https://raw.githubusercontent.com/daenuprobst/covid19-cases-switzerland/master/covid19_cases_switzerland_openzh.csv",sep=",")
df_deaths = pd.read_table("https://raw.githubusercontent.com/daenuprobst/covid19-cases-switzerland/master/covid19_fatalities_switzerland_openzh.csv",sep=",")
df_hosps = pd.read_table("https://raw.githubusercontent.com/daenuprobst/covid19-cases-switzerland/master/covid19_hospitalized_total_switzerland_openzh.csv",sep=",")
df_re = pd.read_table("https://raw.githubusercontent.com/covid-19-Re/dailyRe-Data/master/CHE-confCasesSWestimates.csv",sep=",")

# (side note) Build custom dataset
This part will not be used as the project specifications require the use of classes. And this part of the project seemed to be compatible with that specification. Since I was not planning initially to use classes, this is how it would have been done using a dataframe.

- As we have multiple datasets, they will be merged into 1 custom dataset
- 6 datasets were gathered (from openZH, foph-covid19-data, and dailyRe-Data), they have 2 features in commun: the Date and the Canton. The merge will based on those 2 features.
- As the datasets are coming from different sources, the labeling are not always the same, it must be unified
  - As the name of the Canton column is not always the same, it will be unified as "region"
  - As the name of the Dates column is not always the same, it will be unified as "date"
  - The region value for Switzerland, will be unified as CH
  - Unify the usage of the region column region 
  - It will be only kept the rows present in all datasets for the 2 features.
- Some features are meaningless for our project, so we will drop them: 

| Field Name               | Description                                                                          | Format     | Example                |
|--------------------------|--------------------------------------------------------------------------------------|------------|------------------------|
| date                     | Date of notification                                                                 | YYYY-MM-DD | 2021-02-27             |
| region                   | Abbreviation of the reporting canton                                                 | Text       | NE                     |
| tested                   | Total of Tests until this date                                                       | Value      | 11.0                   |
| tested_diff              | Current date new tests                                                               | Value      | 5.0                    |
| tested_pc                | Percent tests of the canton population                                               | Value      | 1.2878435761777038e-06 |
| tested_diff_pc           | Current date percent tests of the canton population                                  | Value      | 5.85383443717138e-07   |
| cases                    | Total of Confirmed positive cases until this date                                    | Value      | 11.0                   |
| cases_diff               | Current date new cases                                                               | Value      | 5.0                    |
| cases_pc                 | Percent cases of the canton population                                               | Value      | 1.2878435761777038e-06 |
| cases_diff_pc            | Current date percent cases of the canton population                                  | Value      | 5.85383443717138e-07   |
| deaths                   | Total of Deaths until this date                                                      | Value      | 0.0                    |
| deaths_diff              | Current date new deaths                                                              | Value      | 0.0                    |
| deaths_pc                | Percent deaths of the canton population                                              | Value      | 0.0                    |
| deaths_diff_pc           | Current date percent deaths of the canton population                                 | Value      | 0.0                    |
| hospitalizations         | Total of Hospitalizations until this date                                            | Value      | 0.0                    |
| hospitalizations_diff     | Current date new Hospitalizations                                                    | Value      | 0.0                    |
| hospitalizations_pc      | Percent Hospitalization of the canton population                                     | Value      | 0.0                    |
| hospitalizations_diff_pc | Current date percent Hospitalization of the canton population                        | Value      | 0.0                    |
| median_R_mean            | Median R_e                                                                           | Value      | 1.68                   |
| median_R_highHPD         | High Median R_e                                                                      | Value      | 3.32                   |
| median_R_lowHPD          | Low Median R_e                                                                       | Value      | 0.59                   |
| deliveredTotal           | sum of vaccines delivered to the canton till the date                                | Number     | 21000                  |
| deliveredPer100          | sum of vaccines delivered to the canton till the date per 100 population             | Number     | 11.9                   |
| administeredTotal        | number of vaccines vaccinated in the canton till the date                            | Number     | 15955                  |
| administeredPer100       | number of vaccines vaccinated in the canton till the date per 100 population         | Number     | 9.04                   |
| fullyVaccinatedTotal     | number of people who are fully vaccinated (2 doses at the moment)                    | Number     | 2572                   |
| fullyVaccinatedPer100    | number of people who are fully vaccinated (2 doses at the moment) per 100 population | Number     | 1.46                   |


# Making a Class for Regions
I would not recommend this approach expect in a pedagological context, such as this one, as it's clearly not optimized, however it's a still interesting approach.

- Each Region object is initialized with a canton abreviation and a date
- It uses dataframes as global values
- It has only one set method, which is for the date, which will call an internal update method for all internal values
- It has a get method for all internal values
- It has a method returning the list of all internal values in a specific order
- The delivered_diff, administered_diff, fullyVaccinated_diff attributes are calculated by the class

| Region |
|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| +str name <br/>+str date <br/>-float cases <br/>-float cases_diff <br/>-float cases_pc <br/>-float cases_diff_pc <br/>-float deaths <br/>-float deaths_diff <br/>-float deaths_pc <br/>-float deaths_diff_pc <br/>-float hospitalizations <br/>-float hospitalizations_diff <br/>-float hospitalizations_pc <br/>-float hospitalizations_diff_pc <br/>-float median_R_mean <br/>-float median_R_highHPD <br/>-float median_R_lowHPD <br/>-float delivered_diff <br/>-float deliveredTotal <br/>-float deliveredPer100 <br/>-float administered_diff <br/>-float administeredTotal <br/>-float administeredPer100 <br/>-float fullyVaccinated_diff <br/>-float fullyVaccinatedTotal <br/>-float fullyVaccinatedPer100                                                                                                             |
| -updateInternalValues(): void <br/>+get_all(): List<float> <br/>+get_name(): str <br/>+get_date(): str <br/>+get_cases: float <br/>+get_cases_diff: float <br/>+get_cases_pc: float <br/>+get_cases_diff_pc: float <br/>+get_deaths: float <br/>+get_deaths_diff: float <br/>+get_deaths_pc: float <br/>+get_deaths_diff_pc: float <br/>+get_hospitalizations: float <br/>+get_hospitalizations_diff: float <br/>+get_hospitalizations_pc: float <br/>+get_hospitalizations_diff_pc: float <br/>+get_median_R_mean: float <br/>+get_median_R_highHPD: float <br/>+get_median_R_lowHPD: float <br/>+get_delivered_diff: float <br/>+get_deliveredTotal: float <br/>+get_deliveredPer100: float <br/>+get_administered_diff: float <br/>+get_administeredTotal: float <br/>+get_administeredPer100: float <br/>+get_fullyVaccinated_diff: float <br/>+get_fullyVaccinatedTotal: float <br/>+get_fullyVaccinatedPer100: float |

In [4]:
import datetime

class Region:    
    def __init__(self,name:str, date:str = "2021-02-27"):
        self.name = name
        self.date = date
        self.__cases = 0.0
        self.__cases_diff = 0.0
        self.__cases_pc = 0.0
        self.__cases_diff_pc = 0.0
        self.__deaths = 0.0
        self.__deaths_diff = 0.0
        self.__deaths_pc = 0.0
        self.__deaths_diff_pc = 0.0
        self.__hospitalizations = 0.0
        self.__hospitalizations_diff = 0.0
        self.__hospitalizations_pc = 0.0
        self.__hospitalizations_diff_pc = 0.0
        self.__median_R_mean = 0.0
        self.__median_R_highHPD = 0.0
        self.__median_R_lowHPD = 0.0
        self.__delivered_diff = 0.0
        self.__deliveredTotal = 0.0
        self.__deliveredPer100 = 0.0
        self.__administered_diff = 0.0
        self.__administeredTotal = 0.0
        self.__administeredPer100 = 0.0
        self.__fullyVaccinated_diff = 0.0
        self.__fullyVaccinatedTotal = 0.0
        self.__fullyVaccinatedPer100 = 0.0
        
        self.__updateInternalValues()
        
    def __updateInternalValues(self):
        name_diff = self.name+"_diff"
        name_pc = self.name+"_pc"
        name_diff_pc = self.name+"_diff_pc"
        name_date_cases = df_cases.loc[df_cases["Date"] == self.date][[self.name,name_diff,name_pc,name_diff_pc]]
        name_date_deaths = df_deaths.loc[df_deaths["Date"] == self.date][[self.name,name_diff,name_pc,name_diff_pc]]
        name_date_hosps = df_hosps.loc[df_hosps["Date"] == self.date][[self.name,name_diff,name_pc,name_diff_pc]]
        name = "CHE" if self.name == "CH" else self.name
        name_date_re = df_re.loc[(df_re["region"]==name)& (df_re["date"] == self.date)]
        name_date_vacc = df_vacc.loc[(df_vacc["canton"]==self.name) & (df_vacc["date"]==self.date)]    
        last_day = (datetime.datetime.strptime(self.date, '%Y-%m-%d')-datetime.timedelta(1)).strftime("%Y-%m-%d")
        name_date_vacc_previous = df_vacc.loc[(df_vacc["canton"]==self.name) & (df_vacc["date"]==last_day)]
        
        self.__cases = name_date_cases[self.name].item() if not name_date_cases[self.name].empty else 0.0
        self.__cases_diff = name_date_cases[name_diff].item() if not name_date_cases[name_diff].empty else 0.0
        self.__cases_pc = name_date_cases[name_pc].item() if not name_date_cases[name_pc].empty else 0.0
        self.__cases_diff_pc = name_date_cases[name_diff_pc].item() if not name_date_cases[name_diff_pc].empty else 0.0
        self.__deaths = name_date_deaths[self.name].item() if not name_date_deaths[self.name].empty else 0.0
        self.__deaths_diff = name_date_deaths[name_diff].item() if not name_date_deaths[name_diff].empty else 0.0
        self.__deaths_pc = name_date_deaths[name_pc].item() if not name_date_deaths[name_pc].empty else 0.0
        self.__deaths_diff_pc = name_date_deaths[name_diff_pc].item() if not name_date_deaths[name_diff_pc].empty else 0.0
        self.__hospitalizations = name_date_hosps[self.name].item() if not name_date_hosps[self.name].empty else 0.0
        self.__hospitalizations_diff = name_date_hosps[name_diff].item() if not name_date_hosps[name_diff].empty else 0.0
        self.__hospitalizations_pc = name_date_hosps[name_pc].item() if not name_date_hosps[name_pc].empty else 0.0
        self.__hospitalizations_diff_pc = name_date_hosps[name_diff_pc].item() if not name_date_hosps[name_diff_pc].empty else 0.0
        self.__median_R_mean = name_date_re["median_R_mean"].item() if not name_date_re["median_R_mean"].empty else 0.0
        self.__median_R_highHPD = name_date_re["median_R_highHPD"].item() if not name_date_re["median_R_highHPD"].empty else 0.0
        self.__median_R_lowHPD = name_date_re["median_R_lowHPD"].item() if not name_date_re["median_R_lowHPD"].empty else 0.0
        
        self.__deliveredTotal = name_date_vacc["deliveredTotal"].item() if not name_date_vacc["deliveredTotal"].empty else 0.0
        self.__deliveredPer100 = name_date_vacc["deliveredPer100"].item() if not name_date_vacc["deliveredPer100"].empty else 0.0
        self.__administeredTotal = name_date_vacc["administeredTotal"].item() if not name_date_vacc["administeredTotal"].empty else 0.0
        self.__administeredPer100 = name_date_vacc["administeredPer100"].item() if not name_date_vacc["administeredPer100"].empty else 0.0
        self.__fullyVaccinatedTotal = name_date_vacc["fullyVaccinatedTotal"].item() if not name_date_vacc["fullyVaccinatedTotal"].empty else 0.0
        self.__fullyVaccinatedPer100 = name_date_vacc["fullyVaccinatedPer100"].item() if not name_date_vacc["fullyVaccinatedPer100"].empty else 0.0
        
        if not name_date_vacc["deliveredTotal"].empty and not name_date_vacc_previous["deliveredTotal"].empty:
            self.__delivered_diff = name_date_vacc["deliveredTotal"].item() - name_date_vacc_previous["deliveredTotal"].item()
        else: self.__delivered_diff = 0.0
        if not name_date_vacc["administeredTotal"].empty and not name_date_vacc_previous["administeredTotal"].empty:
            self.__administered_diff = name_date_vacc["administeredTotal"].item() - name_date_vacc_previous["administeredTotal"].item()
        else: self.__administered_diff = 0.0
        if not name_date_vacc["fullyVaccinatedTotal"].empty and not name_date_vacc_previous["fullyVaccinatedTotal"].empty:
            self.__fullyVaccinated_diff = name_date_vacc["fullyVaccinatedTotal"].item() - name_date_vacc_previous["fullyVaccinatedTotal"].item()
        else: self.__fullyVaccinated_diff = 0.0
        
        
    def get_all(self):
        return []
    
    def get_name(self):
        return self.name
    
    def get_cases(self):
        return self.__cases
    
    def get_cases_diff(self):
        return self.__cases_diff
    
    def get_cases_pc(self):
        return self.__cases_pc
    
    def get_cases_diff_pc(self):
        return self.__cases_diff_pc
    
    def get_deaths(self):
        return self.__deaths
    
    def get_deaths_diff(self):
        return self.__deaths_diff
    
    def get_deaths_pc(self):
        return self.__deaths_pc
    
    def get_deaths_diff_pc(self):
        return self.__deaths_diff_pc
    
    def get_hospitalizations(self):
        return self.__hospitalizations
    
    def get_hospitalizations_diff(self):
        return self.__hospitalizations_diff
    
    def get_hospitalizations_pc(self):
        return self.__hospitalizations_pc
    
    def get_hospitalizations_diff_pc(self):
        return self.__hospitalizations_diff_pc
    
    def get_median_R_mean(self):
        return self.__median_R_mean
    
    def get_median_R_highHPD(self):
        return self.__median_R_highHPD
    
    def get_median_R_lowHPD(self):
        return self.__median_R_lowHPD
    
    def get_delivered_diff(self):
        return self.__delivered_diff
    
    def get_deliveredTotal(self):
        return self.__deliveredTotal
    
    def get_deliveredPer100(self):
        return self.__deliveredPer100
    
    def get_administered_diff(self):
        return self.__administered_diff
    
    def get_administeredTotal(self):
        return self.__administeredTotal
    
    def get_administeredPer100(self):
        return self.__administeredPer100
    
    def get_fullyVaccinated_diff(self):
        return self.__fullyVaccinated_diff
    
    def get_fullyVaccinatedTotal(self):
        return self.__fullyVaccinatedTotal
    
    def get_fullyVaccinatedPer100(self):
        return self.__fullyVaccinatedPer100
    
region_ch = Region("CH", "2021-02-27")
region_ch.get_fullyVaccinatedTotal()

234993

In [ ]:
#df_cases["AG"][]
#df_cases.loc[df_cases["Date"] == "2021-02-27", "AG"]
#df_cases.loc[df_cases["Date"] == "2021-02-27"]

df_re.loc[(df_re["region"]=="CHE")& (df_re["date"] == "2021-02-26")]

In [ ]:
name = "CH"
name_diff = name+"_diff"
name_pc = name+"_pc"
name_diff_pc = name+"_diff_pc"
name_date = df_hosps.loc[df_cases["Date"] == "2021-02-27"][[name,name_diff,name_pc,name_diff_pc]]
name_date[name].item()
name_date[name_diff].item()
#name_date[name_pc].item()
#name_date[name_diff_pc].item()

In [ ]:
name_date_vacc = df_vacc.loc[(df_vacc["canton"]=="CH") & (df_vacc["date"]=="2021-02-23")]
#name_date_vacc["deliveredTotal"].item() if not name_date_vacc["deliveredTotal"].empty else 0.0

#name_date_vacc["deliveredPer100"].item() if not name_date_vacc["deliveredPer100"].empty else 0.0
#name_date_vacc["administeredTotal"].item() if not name_date_vacc["administeredTotal"].empty else 0.0
#name_date_vacc["administeredPer100"].item() if not name_date_vacc["administeredPer100"].empty else 0.0
#name_date_vacc["fullyVaccinatedTotal"].item() if not name_date_vacc["fullyVaccinatedTotal"].empty else 0.0
#name_date_vacc["fullyVaccinatedPer100"].item() if not name_date_vacc["fullyVaccinatedPer100"].empty else 0.0


In [ ]:
import datetime

date_str = "2020-12-24"
last_day = (datetime.datetime.strptime(date_str, '%Y-%m-%d')-datetime.timedelta(1))

name_date_vacc_previous = df_vacc.loc[(df_vacc["canton"]=="CH") & (df_vacc["date"]==last_day.strftime("%Y-%m-%d"))]

if not name_date_vacc["administeredTotal"].empty and not name_date_vacc_previous["administeredTotal"].empty:
    print(date_str)
    print(last_day)
    print(name_date_vacc["administeredTotal"].item())
    print(name_date_vacc_previous["administeredTotal"].item())
    print(name_date_vacc["administeredTotal"].item() - name_date_vacc_previous["administeredTotal"].item())
else: 
    print(0.0)
    print(last_day < datetime.datetime.strptime("2020-12-23", '%Y-%m-%d'))

In [ ]:
self_name = "AG"
name = "CHE" if self_name == "CH" else self_name
name_date_re = df_re.loc[(df_re["region"]==name)& (df_re["date"] == "2021-02-20")]

print(name_date_re)
#name_date_re = df_re.loc[(df_re["region"]=="CHE")& (df_re["date"] == "2021-02-26")]
#if name == "CHE": name == "CH"
#
if not name_date_re["median_R_mean"].empty: 
    print(name_date_re["median_R_mean"].item())